In [1]:
#Iniziamo

In [2]:
!pip install transformers datasets torch torchvision

from transformers import SegformerFeatureExtractor, SegformerForSemanticSegmentation
from PIL import Image
import requests
import torch
import matplotlib.pyplot as plt

import os
import csv
from torch.utils.data import Dataset
import numpy as np
from torchvision.transforms import Compose, Resize, Normalize
from torchvision.transforms.functional import InterpolationMode


from torchvision import transforms
from transformers import SegformerForSemanticSegmentation, SegformerFeatureExtractor

import numpy as np

from transformers import SegformerFeatureExtractor, SegformerForSemanticSegmentation
import torch

from tqdm import tqdm

from google.colab import drive

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def build_color_to_class_function(base_colors, tolerance=30):
    color_array = np.array(list(base_colors.values()))
    color_labels = list(base_colors.keys())

    def color_to_class(rgb):
        rgb = np.array(rgb)
        dists = np.linalg.norm(color_array - rgb, axis=1)
        min_idx = np.argmin(dists)
        if dists[min_idx] <= tolerance:
            return min_idx
        else:
            return 255

    return color_to_class

In [5]:
base_colors = {
    "white": (255, 255, 255),
    "red": (255, 0, 0),
    "green": (0, 255, 0),
    "blue": (0, 0, 255),
    "cyan": (0, 255, 255)
}
colormap = np.array(list(base_colors.values()), dtype=np.uint8)

In [6]:
color_to_class_fn = build_color_to_class_function(base_colors, tolerance=30)

In [7]:
def decode_segmentation(mask, colormap):
    return colormap[mask]

In [11]:
def encode_segmentation(mask_rgb, colormap):
    mask_rgb = np.array(mask_rgb)
    class_mask = np.zeros(mask_rgb.shape[:2], dtype=np.int64)

    color_to_class = {tuple(color): idx for idx, color in enumerate(colormap)}

    for color, class_idx in color_to_class.items():
        matches = np.all(mask_rgb == color, axis=-1)
        class_mask[matches] = class_idx

    return torch.from_numpy(class_mask).long()

In [8]:
def encode_segmentation_fast(mask, color_to_class_fn):
    mask_np = np.array(mask)
    flat_mask = mask_np.reshape(-1, 3)
    encoded_flat = np.array([color_to_class_fn(tuple(px)) for px in flat_mask], dtype=np.uint8)
    encoded = encoded_flat.reshape(mask_np.shape[:2])
    return torch.from_numpy(encoded)

In [16]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

def get_image_transform():
    return transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])


def get_mask_transform():
    return transforms.Compose([
        transforms.Resize((64, 64), interpolation=Image.NEAREST)
    ])

img_transform = get_image_transform()
mask_transform = get_mask_transform()

In [19]:
backbone = "nvidia/segformer-b3-finetuned-ade-512-512"

model = SegformerForSemanticSegmentation.from_pretrained(
    backbone,
    num_labels=5,
    ignore_mismatched_sizes=True
)

state_dict = torch.load("/content/drive/MyDrive/segformer_cvusa_seg6.pth", map_location="cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(state_dict)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b3-finetuned-ade-512-512 and are newly initialized because the shapes did not match:
- decode_head.classifier.weight: found shape torch.Size([150, 768, 1, 1]) in the checkpoint and torch.Size([5, 768, 1, 1]) in the model instantiated
- decode_head.classifier.bias: found shape torch.Size([150]) in the checkpoint and torch.Size([5]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


SegformerForSemanticSegmentation(
  (segformer): SegformerModel(
    (encoder): SegformerEncoder(
      (patch_embeddings): ModuleList(
        (0): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
          (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        )
        (1): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        )
        (2): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(128, 320, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
        )
        (3): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(320, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)

In [20]:
input_folder = "/content/drive/MyDrive/generated_images"
output_folder = "/content/drive/MyDrive/generated_seg"

os.makedirs(output_folder, exist_ok=True)

transform = get_image_transform()

In [21]:
model.eval()

with torch.no_grad():
    for filename in tqdm(sorted(os.listdir(input_folder))):
        if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        img_path = os.path.join(input_folder, filename)
        img = Image.open(img_path).convert("RGB")
        input_tensor = transform(img).unsqueeze(0).to(device)  # [1, 3, H, W]

        outputs = model(pixel_values=input_tensor)
        preds = outputs.logits.argmax(dim=1).squeeze(0).cpu().numpy()  # [H, W] class indices

        seg_rgb = decode_segmentation(preds, colormap)  # shape: [H, W, 3]
        seg_rgb_img = Image.fromarray(seg_rgb.astype(np.uint8))

        save_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_seg.png")
        seg_rgb_img.save(save_path)

100%|██████████| 2215/2215 [02:53<00:00, 12.75it/s]
